In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

In [2]:
from langchain_community.document_loaders import TextLoader

state_of_the_union_path = '/Users/shivramamurthi/models/state_of_the_union.txt' 
raw_documents = TextLoader(state_of_the_union_path).load()
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)

# splitter.split_text
with open(state_of_the_union_path) as f:
    state_of_the_union_raw_txt = f.read()
state_of_the_union_txts = text_splitter.split_text(state_of_the_union_raw_txt)

# splitter.split_documents
state_of_the_union_docs = text_splitter.split_documents(raw_documents)

print(state_of_the_union_txts[0])
print(state_of_the_union_docs[0])

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.
page_content='Madam Speaker, Madam Vice President, our First

In [3]:
embeddings = SentenceTransformerEmbeddings(model_name='bert-base-nli-mean-tokens')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
faiss1 = FAISS.from_texts(
    state_of_the_union_txts, embeddings, metadatas=[
        {
            "doc_id": str(i),
            "source_name": state_of_the_union_path
        } for i in range(len(state_of_the_union_txts))
    ]
)

In [7]:
chroma1 = Chroma.from_texts(
    state_of_the_union_txts, embeddings, metadatas=[
        {
            "doc_id": str(i),
            "source_name": state_of_the_union_path
        } for i in range(len(state_of_the_union_txts))
    ]
)

In [9]:
while True:
    query = input("[FAISS] Type your search (type '#quit' to stop): ")
    if query == '#quit':
        break
    results = faiss1.similarity_search_with_score(query, k=3)
    for r, score in results:
        content, id, source_name = r.page_content, r.metadata['doc_id'], r.metadata['source_name']
        print(f"Query: '{query}'\n=== Match Id: {id}, Source: {source_name}, Score: {score}, Content:\n{content}")

Query: 'What is our responsibility as the United States?'
=== Match Id: 30, Source: /Users/shivramamurthi/models/state_of_the_union.txt, Score: 311.053466796875, Content:
In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown 

In [13]:
while True:
    query = input("[CHROMA] Type your search (type '#quit' to stop): ")
    if query == '#quit':
        break
    results = chroma1.similarity_search_with_score(query, k=3)
    for r, score in results:
        content, id, source_name = r.page_content, r.metadata['doc_id'], r.metadata['source_name']
        print(f"Query: '{query}'\n=== Match Id: {id}, Source: {source_name}, Score: {score}, Content:\n{content}")

Query: 'Vladimir Putin'
=== Match Id: 30, Source: /Users/shivramamurthi/models/state_of_the_union.txt, Score: 333.06805419921875, Content:
In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top